In [8]:
!pip install pandas psycopg2-binary nltk textblob


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import psycopg2 as pg
from textblob import TextBlob
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DEll\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
conn = pg.connect("dbname='clever' user='clever' host='localhost' port='5433' password='clever'")

In [25]:
query = """
SELECT 
    cp.google_id, 
    cp.name, 
    cp.city, 
    cp.state, 
    AVG(cr.rating) AS avg_rating, 
    COUNT(cr.review_id) AS total_reviews, 
    fs.num_of_trucks, 
    fs.num_of_tractors,
    STRING_AGG(cr.review_text, ' ') AS reviews_text  -- Agrupar todos os textos das avaliações
FROM 
    company_profiles_google_maps cp
LEFT JOIN 
    customer_reviews_google cr ON cp.google_id = cr.google_id
LEFT JOIN 
    fmcsa_company_snapshot fs ON LOWER(cp.name) = LOWER(fs.company_name)
GROUP BY 
    cp.google_id, cp.name, cp.city, cp.state, fs.num_of_trucks, fs.num_of_tractors
ORDER BY 
    avg_rating DESC, fs.num_of_trucks DESC;
"""


In [27]:
df = pd.read_sql(query, conn)

df.head()

C:\Users\DEll\AppData\Local\Temp\ipykernel_19316\3004567250.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,google_id,name,city,state,avg_rating,total_reviews,num_of_trucks,num_of_tractors,reviews_text
0,0x864ea1f1e49601f7:0x65da11c30fd03d2f,"Matt Twomey, Highlands Real Estate",Dallas,Texas,NaN,0,NaN,NaN,None
1,0x14e179155727173b:0x2c342e5f13a9b2d,New Hope Realty,Dallas,Georgia,NaN,0,NaN,NaN,None
2,0x14e4b9f6c46524a9:0x202749ab2be6bfe3,Harvey Mclean & Associates,Dallas,Texas,NaN,0,NaN,NaN,None
3,0x1803389f251373:0x26e9e6cc7312627c,Yanet Mino Realtor,Miami,Florida,NaN,0,NaN,NaN,None
4,0x18751cf7cb7e9a5:0xad85fc0dfe9ff490,Lunabase Travelstays & Property Management,Miami,Florida,NaN,0,NaN,NaN,None


In [28]:
df['ranking_score'] = df['avg_rating'] * df['total_reviews']  
df = df.sort_values(by='ranking_score', ascending=False)  

df[['name', 'city', 'state', 'avg_rating', 'total_reviews', 'ranking_score']].head(10)


,name,city,state,avg_rating,total_reviews,ranking_score
5835,Wildcat Movers - Dallas,Dallas,Texas,4.9,1692,8290.8
5837,Eco Movers Moving & Storage,Seattle,Washington,4.8,1500,7200.0
5781,Every Door Real Estate,Seattle,Washington,5.0,811,4055.0
5854,AB Moving,Dallas,Texas,4.5,724,3258.0
5845,Wrightway Moving Company LLC,Dallas,Texas,4.7,688,3233.6
5783,Pure Moving Company Seattle Movers Local & Lon...,Seattle,Washington,5.0,597,2985.0
5810,Pure Moving Company Miami Movers Local & Long ...,Miami,Florida,5.0,530,2650.0
5856,AM Moving Company,Dallas,Texas,4.4,434,1909.6
5853,Exodus Moving & Storage LLC,Dallas,Texas,4.6,414,1904.4
5822,Phoenix Express Specialty Moving & Delivery,Dallas,Texas,4.9,375,1837.5


In [33]:
miami_df = df[df['city'] == 'Dallas']
miami_df

,google_id,name,city,state,avg_rating,total_reviews,num_of_trucks,num_of_tractors,reviews_text,ranking_score,sentiment
5835,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,Wildcat Movers - Dallas,Dallas,Texas,4.9,1692,NaN,NaN,Hi Wildcat! Itâs a wonder that an agreed upo...,8290.8,0.305222
5854,0x864e99d71a97da87:0xacad6516b3a8297,AB Moving,Dallas,Texas,4.5,724,NaN,NaN,One of the workers came into my house while I ...,3258.0,0.226706
5845,0x864c23dad897733d:0x2de847b08be4e483,Wrightway Moving Company LLC,Dallas,Texas,4.7,688,NaN,NaN,"These guys are awesome, skilled, punctual and ...",3233.6,0.247905
5856,0x864e9c8c3f1dc5eb:0x5afc911aca2e605e,AM Moving Company,Dallas,Texas,4.4,434,NaN,NaN,Daryl and his crew were so nice helped me move...,1909.6,0.226813
5853,0x864e9eee5c772cd5:0x3e7eeeda5db6ca0d,Exodus Moving & Storage LLC,Dallas,Texas,4.6,414,NaN,NaN,Juan and his crew were super fast with a last ...,1904.4,0.257131
...,...,...,...,...,...,...,...,...,...,...,...
5761,0xa8f6b72243a3f1db:0x3c9780f340494dde,TL&G Real Estate,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5763,0xaabd8a8c68d7c89:0xfcf52553324dc2f0,Laura Lopez | Eco Agent Realty International,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5764,0xaacb6de578b99e6b:0x7483db8fc2925e63,Ellis Home Buyers,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5770,0xadda9f4104700271:0xcb498dbaf4e58e18,JacksonSells Team,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN


In [34]:
florida_df = df[df['state'] == 'Texas']
florida_df

,google_id,name,city,state,avg_rating,total_reviews,num_of_trucks,num_of_tractors,reviews_text,ranking_score,sentiment
5835,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,Wildcat Movers - Dallas,Dallas,Texas,4.9,1692,NaN,NaN,Hi Wildcat! Itâs a wonder that an agreed upo...,8290.8,0.305222
5854,0x864e99d71a97da87:0xacad6516b3a8297,AB Moving,Dallas,Texas,4.5,724,NaN,NaN,One of the workers came into my house while I ...,3258.0,0.226706
5845,0x864c23dad897733d:0x2de847b08be4e483,Wrightway Moving Company LLC,Dallas,Texas,4.7,688,NaN,NaN,"These guys are awesome, skilled, punctual and ...",3233.6,0.247905
5856,0x864e9c8c3f1dc5eb:0x5afc911aca2e605e,AM Moving Company,Dallas,Texas,4.4,434,NaN,NaN,Daryl and his crew were so nice helped me move...,1909.6,0.226813
5853,0x864e9eee5c772cd5:0x3e7eeeda5db6ca0d,Exodus Moving & Storage LLC,Dallas,Texas,4.6,414,NaN,NaN,Juan and his crew were super fast with a last ...,1904.4,0.257131
...,...,...,...,...,...,...,...,...,...,...,...
5761,0xa8f6b72243a3f1db:0x3c9780f340494dde,TL&G Real Estate,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5763,0xaabd8a8c68d7c89:0xfcf52553324dc2f0,Laura Lopez | Eco Agent Realty International,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5764,0xaacb6de578b99e6b:0x7483db8fc2925e63,Ellis Home Buyers,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN
5770,0xadda9f4104700271:0xcb498dbaf4e58e18,JacksonSells Team,Dallas,Texas,NaN,0,NaN,NaN,None,NaN,NaN


In [35]:
from textblob import TextBlob

def sentiment_analysis(text):
    if pd.notna(text):
        return TextBlob(text).sentiment.polarity
    return None

df['sentiment'] = df['reviews_text'].apply(sentiment_analysis)
df[['name', 'city', 'state', 'avg_rating', 'total_reviews', 'sentiment']].head()


,name,city,state,avg_rating,total_reviews,sentiment
5835,Wildcat Movers - Dallas,Dallas,Texas,4.9,1692,0.305222
5837,Eco Movers Moving & Storage,Seattle,Washington,4.8,1500,0.293412
5781,Every Door Real Estate,Seattle,Washington,5.0,811,0.296623
5854,AB Moving,Dallas,Texas,4.5,724,0.226706
5845,Wrightway Moving Company LLC,Dallas,Texas,4.7,688,0.247905
